In [1]:
import pandas as pd
import numpy as np

# model_deployment
The purpose of the data product is to provide a suitable output to be analyzed by experts and further enhance the results of the research project.

In [15]:
covid19_global_data_pred = pd.read_csv('covid19_global_data_pred.csv')

In [16]:
# Analyze correlation for cluster0
dut_cluster0_corr = covid19_global_data_pred[covid19_global_data_pred['cluster']==0].iloc[:,106:].corr()
dut_cluster0_corr = dut_cluster0_corr.unstack().sort_values(ascending=False).drop_duplicates().reset_index()
dut_cluster0_corr.rename(columns={0:'cluster0'}, inplace=True)

In [17]:
# Analyze correlation for cluster1
dut_cluster1_corr = covid19_global_data_pred[covid19_global_data_pred['cluster']==1].iloc[:,106:].corr()
dut_cluster1_corr = dut_cluster1_corr.unstack().sort_values(ascending=False).drop_duplicates().reset_index()
dut_cluster1_corr.rename(columns={0:'cluster1'}, inplace=True)

In [18]:
dut_cluster_corr = pd.merge(dut_cluster0_corr, dut_cluster1_corr, how='inner')

In [19]:
# Analyze correlation for cluster2
dut_cluster2_corr = covid19_global_data_pred[covid19_global_data_pred['cluster']==2].iloc[:,106:].corr()
dut_cluster2_corr = dut_cluster2_corr.unstack().sort_values(ascending=False).drop_duplicates().reset_index()
dut_cluster2_corr.rename(columns={0:'cluster2'}, inplace=True)

In [20]:
dut_cluster_corr = pd.merge(dut_cluster_corr, dut_cluster2_corr, how='inner')

In [21]:
# Analyze correlation for cluster3
dut_cluster3_corr = covid19_global_data_pred[covid19_global_data_pred['cluster']==3].iloc[:,106:].corr()
dut_cluster3_corr = dut_cluster3_corr.unstack().sort_values(ascending=False).drop_duplicates().reset_index()
dut_cluster3_corr.rename(columns={0:'cluster3'}, inplace=True)

In [22]:
dut_cluster_corr = pd.merge(dut_cluster_corr, dut_cluster3_corr, how='inner')

In [23]:
# add statistics from the correlated values
dut_cluster_corr['std'] = dut_cluster_corr.iloc[:,2:].apply(lambda x: pd.Series(np.std(x)), axis=1)
dut_cluster_corr['mean'] = dut_cluster_corr.iloc[:,2:-1].apply(lambda x: pd.Series(np.mean(x)), axis=1)

In [24]:
dut_cluster_corr

,level_0,level_1,cluster0,cluster1,cluster2,cluster3,std,mean
0,PA.NUS.ATLS,PA.NUS.FCRF,0.999939,1.000000,1.000000,0.925183,3.238813e-02,0.981281
1,NY.GNP.MKTP.CN,NY.GDP.MKTP.CN,0.999773,0.999930,0.999908,0.999978,7.596496e-05,0.999897
2,NY.GDP.FCST.KD,NY.GDP.MKTP.KD,0.999643,0.999673,0.998770,0.984862,6.289175e-03,0.995737
3,BX.TRF.PWKR.CD.DT,BX.TRF.PWKR.CD,0.999575,0.247510,0.972395,0.998475,3.217556e-01,0.804489
4,BX.TRF.CURR.CD,BN.TRF.CURR.CD,0.999504,-0.925374,0.924138,0.925055,8.124486e-01,0.480831
...,...,...,...,...,...,...,...,...
15107,SP.POP.0014.TO.ZS,SP.POP.5054.FE.5Y,-0.922668,-0.377199,-0.842303,-0.949562,2.318598e-01,-0.772933
15108,SP.DYN.TO65.MA.ZS,SP.DYN.AMRT.FE,-0.946779,-0.920866,-0.889574,-0.955081,2.556101e-02,-0.928075
15109,BX.PEF.TOTL.CD.WD,IP.TMK.RESD,-0.983619,0.391462,-0.443590,0.984717,7.560650e-01,-0.012758
15110,BM.GSR.FCTY.CD,BN.GSR.FCTY.CD,-0.995736,0.569504,-0.731178,-0.739743,6.119375e-01,-0.474288


In [28]:
# define a function that retrieves the top5 correlated and top5 uncorrelated indicators
def indicators_result(str_cluster, std_min, mean_min):
    """
    str_cluster: string cluster under analysis
    std_min: minimum std dev accepted to retrieve information (helps to avoid overfiting)
    mean_min: minimum mean value to retrieve information (helps to avoid overfitting)
    """
    dut_cluster = dut_cluster_corr.sort_values(by=[str_cluster], ascending=[False])
    dut_cluster = dut_cluster[(dut_cluster['std']>std_min)&(np.abs(dut_cluster['mean'])>mean_min)].iloc[np.r_[0:5, -5:0]]
    return(dut_cluster[['level_0','level_1',str_cluster,'std','mean']])

## data product

In [29]:
# results for cluster0 most and least correlated indicators 
indicators_result('cluster0',0.5,0.2)

,level_0,level_1,cluster0,std,mean
4,BX.TRF.CURR.CD,BN.TRF.CURR.CD,0.999504,0.812449,0.480831
38,DT.NFL.PRVT.CD,DT.NFL.PBND.CD,0.979082,0.840391,0.494935
50,IP.TMK.RSCT,MS.MIL.MPRT.KD,0.974458,0.511677,0.276962
91,NV.AGR.TOTL.CD,BN.TRF.CURR.CD,0.947876,0.638052,0.389949
129,NY.GDP.MKTP.CD,BN.TRF.CURR.CD,0.928929,0.752850,0.260616
15074,BN.GSR.FCTY.CD,BM.GSR.TOTL.CD,-0.831240,0.644542,-0.360252
15082,SP.POP.DPND,SP.POP.7579.FE.5Y,-0.847739,0.586223,-0.223984
15094,BN.GSR.FCTY.CD,SH.DTH.MORT,-0.868540,0.540636,-0.263719
15097,NE.RSB.GNFS.CD,COVID19.MEA,-0.871296,0.524853,-0.302076
15110,BM.GSR.FCTY.CD,BN.GSR.FCTY.CD,-0.995736,0.611937,-0.474288


In [30]:
# results for cluster1 most and least correlated indicators 
indicators_result('cluster1',0.5,0.2)

,level_0,level_1,cluster1,std,mean
294,DT.TDS.DIMF.CD,IP.TMK.TOTL,0.999984,0.552857,0.384264
7461,NE.RSB.GNFS.KN,NV.AGR.TOTL.CN,0.999925,0.501385,0.497143
12185,NE.RSB.GNFS.KN,NV.IND.MANF.KN,0.999633,0.507023,0.461504
310,IP.TMK.RESD,DT.DOD.PVLX.CD,0.999393,0.583246,0.520502
13638,NE.EXP.GNFS.KN,NE.RSB.GNFS.KN,0.999140,0.741928,0.271346
10572,NY.GDP.FCST.CN,DT.NFL.PCBK.CD,-0.999994,0.564001,-0.340047
8773,NY.TAX.NIND.KN,DT.NFL.PCBK.CD,-0.999998,0.576515,-0.432248
11361,FM.LBL.BMNY.CN,DT.NFL.PCBK.CD,-0.999999,0.507120,-0.500106
13120,NY.GDP.MKTP.KN,DT.NFL.PCBK.CD,-0.999999,0.556337,-0.475097
9546,NV.IND.TOTL.KN,DT.NFL.PCBK.CD,-1.000000,0.582444,-0.432355


In [31]:
# results for cluster2 most and least correlated indicators 
indicators_result('cluster2',0.5,0.2)

,level_0,level_1,cluster2,std,mean
2473,BX.GSR.ROYL.CD,IT.MLT.MAIN,0.990207,0.514368,0.456771
2703,IP.JRN.ARTC.SC,BX.GSR.ROYL.CD,0.989518,0.521047,0.465672
38,DT.NFL.PRVT.CD,DT.NFL.PBND.CD,0.985618,0.840391,0.494935
5126,IS.AIR.GOOD.MT.K1,NY.GNP.MKTP.CD,0.972501,0.557113,0.395796
4988,NY.GNP.ATLS.CD,IS.AIR.GOOD.MT.K1,0.972408,0.555715,0.397434
14557,SL.TLF.TOTL.IN,BN.KLT.DINV.CD,-0.815330,0.652192,-0.216548
1698,NE.RSB.GNFS.KN,NE.RSB.GNFS.CN,-0.849420,0.753202,0.372930
14965,NY.GNP.MKTP.CD,BN.KAC.EOMS.CD,-0.953293,0.647213,-0.497171
15061,BN.KAC.EOMS.CD,BX.PEF.TOTL.CD.WD,-0.972264,0.521374,-0.546952
10311,IP.TMK.RSCT,BN.KAC.EOMS.CD,-0.982472,0.587400,-0.419134


In [32]:
# results for cluster3 most and least correlated indicators 
indicators_result('cluster3',0.5,0.2)

,level_0,level_1,cluster3,std,mean
1098,FM.AST.NFRG.CN,NY.TRF.NCTR.CN,0.999992,0.800128,0.357505
221,FM.LBL.BMNY.CN,NY.GDP.DISC.CN,0.999985,0.637821,0.526722
14793,NV.IND.TOTL.KD,BN.GSR.GNFS.CD,0.999935,0.540028,0.377548
13638,NE.EXP.GNFS.KN,NE.RSB.GNFS.KN,0.999860,0.741928,0.271346
14835,NY.GDP.DISC.KN,NE.RSB.GNFS.KN,0.999853,0.628326,0.565926
13120,NY.GDP.MKTP.KN,DT.NFL.PCBK.CD,-0.999083,0.556337,-0.475097
9546,NV.IND.TOTL.KN,DT.NFL.PCBK.CD,-0.999505,0.582444,-0.432355
1152,IC.LGL.DURS,FI.RES.TOTL.DT.ZS,-0.999854,0.589121,-0.339796
12929,NE.CON.PRVT.CD,IC.TAX.LABR.CP.ZS,-0.999963,0.516346,-0.216797
11361,FM.LBL.BMNY.CN,DT.NFL.PCBK.CD,-0.999980,0.507120,-0.500106
